In [1]:
import polars as pl
from pathlib import Path

# ── 원본 파일 경로 ─────────────────────────────────────────
src = Path("~/Downloads/SKSNXTRDIJH_2023_12.dat").expanduser()

# 헤더 리스트 (질문에서 그대로 사용)
column_names = [
    "일자","대량매매구분코드","정규시간외구분코드","종목코드","종목인덱스","체결번호","체결가격","체결수량",
    "체결유형코드","체결일자","체결시각","근월물체결가격","원월물체결가격","매수회원번호","매수호가유형코드",
    "매수자사주신고서ID","매수자사주매매방법코드","매수매도유형코드","매수위탁자기구분코드","매수위탁사번호",
    "매수PT구분코드","매수투자자구분코드","매수외국인투자자구분코드","매수호가접수번호","매도회원번호",
    "매도호가유형코드","매도자사주신고서ID","매도자사주매매방법코드","매도매도유형코드","매도위탁자기구분코드",
    "매도위탁사번호","매도PT구분코드","매도투자자구분코드","매도외국인투자자구분코드","매도호가접수번호",
    "시가","고가","저가","직전가격","누적체결수량","누적거래대금","최종매도매수구분코드","LP보유수량",
    "데이터구분","메세지일련번호","매수프로그램호가신고구분코드","매도프로그램호가신고구분코드","보드ID",
    "세션ID","실시간상한가","실시간하한가"
]

# ── 최종 CSV에 남길 컬럼 ──────────────────────────────────
keep_cols = ["종목코드","종목인덱스","체결가격","체결일자","체결시각","시가"]

# ── LazyFrame 스트리밍 파이프라인 ─────────────────────────
scan = (
    pl.scan_csv(
        src,
        separator="|",
        new_columns=column_names,
        ignore_errors=True,        # 일부 컬럼 누락 라인은 NaN
        infer_schema_length=500,   # 앞 500줄만 타입 추론
        has_header=False,          # 원본 파일에 헤더 없다고 가정
        low_memory=True            # 메모리 절약 모드
    )
    .select(keep_cols)             # 필요한 열만 선택
)



In [2]:
# ── 디스크로 바로 스트리밍 저장 ───────────────────────────
dst = "SKSNXTR_filtered.csv"
scan.sink_csv(
    dst,
    include_header=True,   # 헤더 작성
    batch_size=1_000_000   # 100만 row 단위로 flush (필요하면 조정)
)

print(f"✔️  saved: {dst}")

✔️  saved: SKSNXTR_filtered.csv
